In this notebook, we will quantify the running time of each strategy (except the ML strategy which will be measured in a different notebook). We will be using the same system parameters and we will measure the time for a single case scenario, i.e. for a given channel matrix **H** and power matrix **P**, how much time in seconds will it take to generate the sets of users belonging to group 1 and group 2, respectively. 

We start by recalling the parameters of the system model.

In [1]:
N = 4 # number of antennas in the BS
K = 10 # number of users (K >= N)
X0 = 0.500 # parameter of the uniform distribution of the user on the x-axis
Y0 = 0.500 # parameter of the uniform distribution of the user on the y-axis
nsize = 10000 # size of the dataset

In [2]:
from itertools import product, combinations
import numpy as np
from numpy.linalg import inv
import h5py
import pandas as pd
from PIL import Image
from numpy import linalg as LA
import os
from sklearn.model_selection import train_test_split
from PIL import Image
from matplotlib.pyplot import imshow
import seaborn as sns
from sklearn import metrics
from IPython.display import clear_output
%matplotlib inline
np.random.seed(42)

Now, we will read the matrices **H** and **P** from the test set (stored in the `test_Data` folder).

In [3]:
with h5py.File('test_Data/H_1.h5', 'r') as hf:
    H = hf['H_1'][:]
with h5py.File('test_Data/P_1.h5', 'r') as hf:
    P = hf['P_1'][:]

In [4]:
H

array([[ 1.        +0.j        ,  1.        +0.j        ,
         1.        +0.j        ,  1.        +0.j        ,
         1.        +0.j        ,  1.        +0.j        ,
         1.        +0.j        ,  1.        +0.j        ,
         1.        +0.j        ,  1.        +0.j        ],
       [-0.94840597-0.31705854j, -0.99435984+0.10605902j,
         0.30158506+0.95343928j, -0.95887464-0.28382993j,
         0.21130964-0.97741917j,  0.27610195+0.96112836j,
        -0.32671863-0.94512165j, -0.58449918-0.8113943j ,
        -0.40006207-0.91648805j, -0.986151  -0.16584992j],
       [ 0.79894777+0.60140042j,  0.97750297-0.21092166j,
        -0.81809291+0.57508608j,  0.83888114+0.54431464j,
        -0.91069647-0.41307619j, -0.84753543+0.53073882j,
        -0.78650987+0.61757771j, -0.31672141+0.94851861j,
        -0.67990068+0.73330421j,  0.94498761+0.32710613j],
       [-0.56704769-0.82368496j, -0.94961955+0.31340504j,
        -0.79503425-0.60656454j, -0.64988907-0.76002908j,
        -0.

The matrix **P** is stored as a whole matrix whereas in the code used to determine the strategies, we only need the diagonal elements, i.e. **P** is treated as a vector.

In [5]:
P = np.diag(P)

In [6]:
P

array([2.29947332e+00, 1.67157783e+04, 2.57658818e+02, 8.23618109e+02,
       5.37619507e+02, 6.21853995e+00, 3.86499865e+01, 1.68015618e+01,
       7.49921433e+01, 1.31926327e+02])

# 1. Exhaustive Search

The first strategy to time is the exhaustive search, we recall the functions used to compute the set of users of group 1 and group 2.

In [7]:
def clusters(A):
    subsets = []
    for i in range(1, len(A)):
        subsets.append(list(combinations(A,i)))
    combos = []
    for i in range(1, int(1+len(subsets)/2)):
        combos.extend(list(product(subsets[i-1], subsets[-i])))
    if not len(A) % 2:
        combos.extend(list(combinations(subsets[int(len(A)/2)-1], 2)))
    return [combo for combo in combos if not set(combo[0]) & set(combo[1])]

In [8]:
def hermetian(H):
    return H.conj().T

In [9]:
def SNR1(H, P, sigma, i):
    K = H.shape[1]
    N = H.shape[0]
    P_i= P[i]
    h_i = H[:,i][:, np.newaxis]
    P = np.delete(P, i)
    set_minus_i = list(range(K))
    set_minus_i.pop(i)
    SNR = P_i*hermetian(h_i).dot(inv(H[:,set_minus_i].dot(np.diag(P)).dot(hermetian(H[:,set_minus_i])) + sigma**2*np.identity(N))).dot(h_i)
    return SNR.real

In [ ]:
def SNR2(H, P, sigma, i, set_I1):
    K = H.shape[1]
    N = H.shape[0]
    P_i= P[i]
    set_0 = set_I1 + [i]
    h_i = H[:,i][:, np.newaxis]
    P = np.delete(P, set_0)
    set_minus_i = list(range(K))
    set_minus_i = [j for j in set_minus_i if j not in set_0]
    SNR = P_i*hermetian(h_i).dot(inv(H[:,set_minus_i].dot(np.diag(P)).dot(hermetian(H[:,set_minus_i])) + sigma**2*np.identity(N))).dot(h_i)
    return SNR.real

In [ ]:
def labelling(N, K, H, P):
    sigma=1
    A = set(range(K))
    set_total = clusters(A)
    set_total.append((tuple(A),()))
    set_1 = [list(set_total[i][0]) for i in range(2**(K-1))]
    set_2 = [list(set_total[i][1]) for i in range(2**(K-1))]
    set_I1 = set_1 + set_2
    set_I2 = set_2 + set_1
    R_1 = np.zeros(2**K)
    R_2 = np.zeros(2**K)
    for i in range(2**K): #each possible scenario
        SNR_1 = [SNR1(H, P, sigma, s) for s in set_I1[i]] # SNR_i^(1) for i in I_1
        SNR_2 = [SNR2(H, P, sigma, s, set_I1[i]) for s in set_I2[i]] # SNR_i^(2) for i in I_2
        R_1[i] = np.sum(np.log(1+np.array(SNR_1)))
        R_2[i] = np.sum(np.log(1+np.array(SNR_2)))
    # define the criterion to select the best configuration (max(R1 + R2) or fairness: max(min(R1,R2)) or max R1*R2)
    sum_rate = R_1 + R_2
    best_index = np.argmax(sum_rate)
    return set_I1[best_index], set_I2[best_index]

Now, we will use the time package to time how much time will it take to find the users of both groups using exhaustive search.

In [ ]:
import time
total = []
for i in range(10):
    t0 = time.time()
    set0, set1 = labelling(N, K, H, P)
    t1 = time.time()
    total.append(t1-t0)

In [ ]:
sum(total)/10

# 2. SNR strategies

For the SNR strategy, we are showing one strategy here but simply modifying the code to take into consideration the other stratgeies will lead to similar CPU time.

In [ ]:
def snr_labelling(N, K, H, P):
    power = []
    for i in range(K):
        h_i = H[:,i][:, np.newaxis]
        P_i = P[i]
        power.append(LA.norm(h_i)**2*P_i)
        
    s = sorted(range(len(power)), key=lambda k: power[k])
                           
    set0 = [s[i] for i in range(int(len(s)/2))]
    set1 = [s[i] for i in range(int(len(s)/2), len(s))]
    return set0, set1

In [ ]:
total = []
for i in range(10):
    t0 = time.time()
    set0, set1 = snr_labelling(N, K, H, P)
    t1 = time.time()
    total.append(t1-t0)

In [ ]:
sum(total)/10

# 3. Random strategy

Finally, we time the random strategy.

In [ ]:
import random 
def partition(list_in, n):
    random.shuffle(list_in)
    l = [list_in[i::n] for i in range(n)]
    return l[0], l[1]

In [ ]:
list_in = [i for i in range(K)]
total = []
for i in range(10):
    t0 = time.time()
    set0, set1 = partition(list_in, 2)
    t1 = time.time()
    total.append(t1-t0)

In [ ]:
sum(total)/10